# Semana 5 — Processos Estocásticos

Este notebook cobre os exercícios propostos: cadeia de Markov simples, simulação de processos de Poisson, comentários de aplicação e um modelo de fila M/M/1.

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(42)

## Cadeia de Markov discreta

Modelamos um sistema de previsão do tempo com dois estados ("Sol" e "Chuva") e analisamos a distribuição estacionária.

In [ ]:
states = ["Sol", "Chuva"]
transition_matrix = np.array([
    [0.8, 0.2],  # Hoje: Sol -> amanhã
    [0.3, 0.7],  # Hoje: Chuva -> amanhã
])

print("Matriz de transição:")
print(pd.DataFrame(transition_matrix, index=states, columns=states))

# simulação de trajetória
num_steps = 20
trajectory = [0]  # começa em Sol
for _ in range(num_steps - 1):
    current = trajectory[-1]
    next_state = np.random.choice([0, 1], p=transition_matrix[current])
    trajectory.append(next_state)

trajectory_labels = [states[i] for i in trajectory]
trajectory_labels

In [ ]:
# cálculo aproximado da distribuição estacionária via potências sucessivas
power_matrix = np.linalg.matrix_power(transition_matrix, 50)
stationary = power_matrix[0]

# solução analítica usando autovetor do valor 1
values, vectors = np.linalg.eig(transition_matrix.T)
stationary_vector = np.real(vectors[:, np.isclose(values, 1)])
stationary_vector = stationary_vector / stationary_vector.sum()
stationary_closed = stationary_vector.flatten()

pd.DataFrame({
    "Estado": states,
    "Distribuição estacionária (simulação)": stationary,
    "Distribuição estacionária (analítica)": stationary_closed,
})

## Processo de Poisson

Simulamos chegadas em um intervalo de tempo contínuo usando tempos entre chegadas exponenciais. Também comparamos o número de eventos por janela com a média teórica ($\lambda t$).

In [ ]:
lam = 3  # taxa média de chegadas por unidade de tempo
horizon = 10

inter_arrival_times = np.random.exponential(scale=1/lam, size=1000)
arrival_times = inter_arrival_times.cumsum()
arrival_times = arrival_times[arrival_times <= horizon]

print(f"Chegadas simuladas no horizonte: {len(arrival_times)}")

window = 1.0
bins = np.arange(0, horizon + window, window)
counts, _ = np.histogram(arrival_times, bins=bins)

fig, ax = plt.subplots(figsize=(8, 4))
ax.bar(bins[:-1], counts, width=0.9, align="edge", edgecolor="black")
ax.axhline(lam * window, color="red", linestyle="--", label="Média teórica λt")
ax.set_xlabel("Tempo")
ax.set_ylabel("Eventos por janela")
ax.set_title("Simulação de processo de Poisson")
ax.legend()
plt.show()

## Processos estocásticos em RL e filas

- **Reforço (RL):** o ambiente de RL é frequentemente modelado como um processo de decisão de Markov (MDP), generalizando cadeias de Markov com recompensas. A dinâmica de transição e a aleatoriedade de recompensas impactam a política ótima.
- **Modelagem de filas:** chegadas e atendimentos aleatórios formam cadeias de Markov ou processos de Poisson (para M/M/1). Métricas como tempo médio no sistema e ocupação dependem da taxa de chegada e de serviço.

## Fila M/M/1: simulação e métricas

Simulamos uma fila com chegadas em processo de Poisson (taxa $\lambda$) e atendimentos com distribuição exponencial (taxa $\mu$). Calculamos:

- Utilização ($\rho = \lambda/\mu$)
- Tempo médio no sistema
- Tempo médio na fila
- Número médio de clientes no sistema

Também comparamos os resultados empíricos com as fórmulas teóricas do modelo M/M/1 (válidas para $\rho < 1$).

In [ ]:
def simulate_mm1(lam: float, mu: float, num_customers: int = 5000):
    arrival_times = np.cumsum(np.random.exponential(scale=1/lam, size=num_customers))
    service_times = np.random.exponential(scale=1/mu, size=num_customers)

    start_service = np.empty(num_customers)
    finish_service = np.empty(num_customers)

    for i in range(num_customers):
        if i == 0:
            start_service[i] = arrival_times[i]
        else:
            start_service[i] = max(arrival_times[i], finish_service[i-1])
        finish_service[i] = start_service[i] + service_times[i]

    waiting_times = start_service - arrival_times
    system_times = finish_service - arrival_times

    return {
        "arrival_times": arrival_times,
        "service_times": service_times,
        "waiting_times": waiting_times,
        "system_times": system_times,
    }


def theoretical_mm1(lam: float, mu: float):
    rho = lam / mu
    assert rho < 1, "A condição de estabilidade (rho < 1) deve ser satisfeita."
    L = rho / (1 - rho)
    Lq = rho**2 / (1 - rho)
    W = 1 / (mu - lam)
    Wq = lam / (mu * (mu - lam))
    return {"rho": rho, "L": L, "Lq": Lq, "W": W, "Wq": Wq}


def summarize_mm1(lam: float, mu: float, num_customers: int = 5000):
    sim = simulate_mm1(lam, mu, num_customers)
    theory = theoretical_mm1(lam, mu)

    empirical = {
        "rho": lam / mu,
        "tempo_medio_sistema": sim["system_times"].mean(),
        "tempo_medio_fila": sim["waiting_times"].mean(),
        "clientes_medio_sistema": (sim["system_times"].mean() * lam),
    }

    summary = pd.DataFrame({
        "Métrica": ["Utilização (rho)", "Tempo médio no sistema (W)", "Tempo médio na fila (Wq)", "Clientes médios no sistema (L)"],
        "Simulação": [empirical["rho"], empirical["tempo_medio_sistema"], empirical["tempo_medio_fila"], empirical["clientes_medio_sistema"]],
        "Teórico": [theory["rho"], theory["W"], theory["Wq"], theory["L"]],
    })
    return summary.round(4)


summary_df = summarize_mm1(lam=2.0, mu=3.5, num_customers=5000)
summary_df

A proximidade entre os valores simulados e teóricos indica que a implementação respeita as propriedades do modelo M/M/1. Alterar $\lambda$ ou $\mu$ ajuda a visualizar como a ocupação cresce quando $\rho$ se aproxima de 1.